In [ ]:
import os

from collections import defaultdict

import logging

from DataLoader import (
    config,
    loader
)

import pandas as pd
import numpy as np

from scipy.optimize import curve_fit

from scipy.interpolate import (
    UnivariateSpline,
    CubicSpline
)

from copy import deepcopy

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import BayesianRidge
from sklearn.preprocessing import (
    MinMaxScaler,
    PolynomialFeatures
)
from sklearn.decomposition import PCA
from sklearn.metrics import (
    mean_squared_error, 
    mean_absolute_error
)

In [ ]:
def compress(signals: pd.DataFrame, floor: str='30min', method='max'):
    """
    floor: https://pandas.pydata.org/docs/user_guide/timeseries.html#timeseries-offset-aliases
    method: Определяет каким образом сжимается ряд. Принимает значения 'max' - максимум интервала, 'mean' - среднее значение интервала, 'mixed' - E(x) + max()
    """
    match method:
        case 'max':
            return signals.groupby(signals.date.dt.floor(floor)).max().drop('date', axis=1)
        case 'mean':
            return signals.groupby(signals.date.dt.floor(floor)).mean().drop('date', axis=1)
        case 'mixed': 
            pass
            # convolve, how to optimize params? perceptron?
        case _:
            raise ValueError(f'Unknown method: {method}')

In [ ]:
data_path = '../data/raw/'

files = os.listdir(data_path)

# make a dict, key - file_name_last part
dta = dict()
datasets = []
for file in files:
    df = pd.read_csv(os.path.join(data_path, file), skiprows=config.COUNT_SKIP, sep=';')
    df = loader.fill_empty(loader.transform_header(df))
    # Compress signal
    compressed = compress(df, floor='10min', method='mean')
    datasets.append(compressed)

min_length = min(df.shape[0] for df in datasets)
min_index = set(datasets[0].index)
for df in datasets[1:]:
    min_index.intersection_update(df.index)
min_index = list(min_index)

for i, df in enumerate(datasets):
    datasets[i] = df.loc[min_index]

# Проверка, что даты совпадают
for i in range(len(files)):
    for j in range(i + 1, len(files)):
        assert np.setdiff1d(datasets[i].index, datasets[j].index).size == 0, f"Intersection has shape {np.setdiff1d(datasets[i].index, datasets[j].index).shape}"

for i in range(len(datasets)):
    datasets[i].sort_index(inplace=True)

for i, file in enumerate(files):

    splitted = loader.split(datasets[i].columns)
    group = loader.group(splitted, datasets[i])

    dta[file] = group

In [ ]:
def exponential_moving_average(data, window):
    weights = np.exp(np.linspace(-1., 0., window))
    weights /= weights.sum()
    ema = np.convolve(data, weights, mode='full')[:len(data)]
    ema[:window] = ema[window]
    return ema

In [ ]:
def idx2time(rng: np.array):
    pass

In [ ]:
fig, ax = plt.subplots(5, 4, figsize=(14, 14))
for i, file in enumerate(files):
    j = 0 
    for k in dta[file].keys():
        for kk in dta[file][k].keys():
            print(i, j, file, k, kk)
            # print(dta[file][k][kk][len(dta[file][k][kk]) // 2:][0])
            if len(dta[file][k][kk]) == 2:
                sns.lineplot(exponential_moving_average(dta[file][k][kk][len(dta[file][k][kk]) // 2:][0], window=50), ax=ax[i][j])
            else:
                sns.lineplot(dta[file][k][kk][len(dta[file][k][kk]) // 2:], ax=ax[i][j])
            # sns.kdeplot(dta[file][k][kk][len(dta[file][k][kk]) // 2:], ax=ax[i][j], color='green')
            # sns.histplot(dta[file][k][kk][len(dta[file][k][kk]) // 2:], ax=ax[i][j])
            # ax[i][j].set_xscale('log')
            # ax[i][j].set_yscale('log')
            ax[i][j].set_title(file + ' ' + k + ' ' + kk, fontsize=8)
            j += 1

Интересные сигналы:
1. Ускорение 10кГц ($y = -\alpha e^{\theta x}$)
2. Эксцесс ($y = b + \alpha e^{\theta x}$)
3. Скорость ($y = b + \alpha e^{\theta x}$)
4. Перемещение ($y = -kx + b$) (возможно, не стоит брать, нужно бустрапить данные и смотреть статистики)

Иначе говоря всё, кроме обычного ускорения

Можно сделать бутстрап по этим сигналам, чтобы получить интервал среднего

объединение данных по компонентам (перенести в лоадер)

In [ ]:
dd = defaultdict(list)
for d in (dta.keys()):
    for k_outer, v_outer in dta[d].items():
        for k_inner, v_inner in v_outer.items():
            # print(k_inner, v_inner)
            dd[k_inner].append(v_inner)

for key in dd.keys():
    component_mat = np.array([])
    for row in dd[key]:
        data_row = np.array(row[len(row) // 2:])
        if component_mat.size == 0:
            component_mat = data_row
        else:
            component_mat = np.vstack([component_mat, data_row])

In [ ]:
def fit_exp_custom(x, f, t, b, e, s):
    return f + t * e ** (b * x + e - (s ** 2) / 2)

In [ ]:
dd['ППД']

In [ ]:
# tmp = np.array(np.arange(1, len(dd['ППД'][0][1]) + 1))
tmp = np.array([])
for i in range(1, 5):
    if tmp.size == 0:
        tmp = dd['ППД'][i][1]
    else:
        tmp = np.vstack([tmp, dd['ППД'][i][1]])

In [ ]:
tmp

Перед PCA нормализуем данные

In [ ]:
def tkeo_operator(data, k = 1):
    """
    Teager-Kaiser Energy operator
    """
    npnts = len(data[0])
    nsignals = len(data)
    filt_data = deepcopy(data)
    for i in range(nsignals):
        for n in range(k, npnts-k):
            filt_data[i][n] = data[i][n]**2-data[i][n-1]*data[i][n+1]
    return filt_data

def normilize(signal: np.ndarray):
    """
    MinMaxScaler + Teager-Kaiser Operator + MinMaxScaler
    """
    # scalers = [MinMaxScaler, StandardScaler]
    scaler = MinMaxScaler(feature_range=(0, 1))
    signal = scaler.fit_transform(signal)
    print(f'norm1 max: {signal.max()}, min: {signal.min()}')
    signal = tkeo_operator(signal)
    print(f'tkeo max: {signal.max()}, min: {signal.min()}')
    signal = scaler.fit_transform(signal)
    print(f'norm2 max: {signal.max()}, min: {signal.min()}')
    return signal


In [ ]:
tmp = normilize(tmp)

In [ ]:
pca = PCA(n_components=2)
compressed = pca.fit_transform(tmp.T)

In [ ]:
sns.lineplot(exponential_moving_average(abs(compressed[:, 1]), window=100))

Интерполяция сигнала слайнами - апостериорная информация не применяется 

In [ ]:
hist = []
tail_len = 500
x = np.arange(len(exponential_moving_average(abs(compressed[:, 1]), window=100)))
y = exponential_moving_average(abs(compressed[:, 1]), window=100)
for elem in x[::500]:
    hist.append(np.arange(elem, elem + tail_len))

for i in range(1, 7):
    uni_spl = UnivariateSpline(np.arange(0, 500 * i), y[np.arange(0, 500 * i)])
    sns.lineplot(x=np.arange(500 * i, len(x)), y=uni_spl(np.arange(500 * i, len(x))))
    len(hist)

sns.lineplot(y, label='true')
plt.yscale('log')
plt.title('Interpolate using univariate spline');

Определяем функцию деградации.  
Интерполяция сигнала экспоненциальной функцией

In [ ]:
def fit_exp_weird(x, f, t, b, e, s):
    return f + t * e ** (b * x + eps - (s ** 2) / 2)
def fit_exp_classic(x, a, b, c):
    return b + a * np.e ** (np.log(x) / c)

In [ ]:
xdata = np.arange(1000)
ydata = exponential_moving_average(abs(compressed[:, 1]), window=100)[:1000]
popt, pcov = curve_fit(fit_exp_custom, xdata, ydata)
popt2, pcov2 = curve_fit(fit_exp_classic, xdata, ydata)

In [ ]:
xdata_apriory = np.arange(1000, 3260)
y_true = exponential_moving_average(abs(compressed[:, 1]), window=100)[xdata_apriory]
y_pred = fit_exp_custom(xdata_apriory, *popt)
y_pred2 = fit_exp_classic(xdata_apriory, *popt2)

plt.plot(y_true, label='True')
plt.plot(y_pred, label='Pred1')
plt.plot(y_pred2, label='Pred2')
plt.legend()
plt.title('Non bayesian predict')

При предсказании на ~570 часов выход на уровень ошибки происходит за ~450 часов.  
Отставание ~125 часов.  
Забустрапить реализацию выборки  

In [ ]:
for i in range(0, 2260):
    delta = abs(y_pred[i] - 0.3765174192095883)
    if delta < .001:
        print(f'Tick: {1000 + i}, dt = {3360 - (1000 + i)}')
        break

Bayesian Ridge

In [ ]:
model = BayesianRidge(max_iter=1000, tol=.001)
model.fit(xdata.reshape(-1, 1), ydata.reshape(-1, 1))

pred, std = model.predict(xdata_apriory.reshape(-1, 1), return_std=True)
fig, ax = plt.subplots(1, 1, figsize=(12, 12))
plt.plot(y_true, label='True')
plt.plot(pred, label='Pred')
plt.scatter(xdata_apriory - 1000, y_true)
plt.fill_between(xdata_apriory - 1000, pred-std, pred+std, color='pink', alpha=0.5, label='pred std')
plt.legend();
plt.title('Default $\\lambda$ and $\\alpha$ init')

In [ ]:
for i in range(0, 2260):
    delta = abs(pred[i] - 0.3765174192095883)
    if delta < .001:
        print(f'Tick: {1000 + i}, dt = {3360 - (1000 + i)}')
        break

Незначительное снижение отставания от реального времени. Это не совсем то, что нужно.

***BAYESIAN INFERENCE***

$P(\theta|X) = \frac{P(\theta) P(X|\theta)}{norm}$

Мы "знаем" изначальное распределение параметров модели

$y(t) = \phi + \theta exp(\beta t + \epsilon - \frac{\sigma^2}{2})$

$\phi$ - const  
$\theta$ - lognorm  

$\beta$ - norm (gaussian)  

ATM SKIP, CANNOT IMPLEMENT, HAVE NO IDEA ABOUT MCMC

In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt
# from scipy.stats import norm, lognorm

# # Функция для вычисления функции y(t) с заданными параметрами
# def y(t, phi, theta, beta):
#     return phi + theta * np.exp(beta * t)

# # Априорные распределения параметров
# prior_theta_mean = 1.0
# prior_theta_std = 0.5
# prior_beta_mean = 0.0
# prior_beta_std = 0.1

# # Начальные значения параметров
# phi = 0.0
# theta = np.random.lognormal(prior_theta_mean, prior_theta_std)
# beta = np.random.normal(prior_beta_mean, prior_beta_std)

# # Временной ряд (первые 100 значений для начальной модели)
# time_series = exponential_moving_average(abs(compressed[:, 1]), window=100)

# # Построение начальной модели на первых 100 значениях временного ряда
# for t, y_value in enumerate(time_series[:1500]):
#     new_t = t + 1
#     new_y = y_value
    
#     # Обновление апостериорных распределений
#     posterior_theta_mean = (prior_theta_mean * prior_theta_std**2 + new_y * np.exp(-beta * new_t) * theta) / (prior_theta_std**2 + np.exp(-2 * beta * new_t))
#     posterior_theta_std = np.sqrt((prior_theta_std**2 * np.exp(2 * beta * new_t)) / (prior_theta_std**2 + np.exp(2 * beta * new_t)))
#     posterior_beta_mean = (prior_beta_mean * prior_beta_std**2 + (new_y - phi - theta * np.exp(-prior_beta_mean * new_t)) * new_t) / (prior_beta_std**2 + new_t**2)
#     posterior_beta_std = np.sqrt(prior_beta_std**2 / (prior_beta_std**2 + new_t**2))

#     # Генерация новых значений параметров из апостериорных распределений
#     theta = np.random.lognormal(posterior_theta_mean, posterior_theta_std)
#     beta = np.random.normal(posterior_beta_mean, posterior_beta_std)

# # Создание массивов для отслеживания изменений параметров
# theta_values = []
# beta_values = []

# # fig, axes = plt.subplots(2, 1, figsize=(10, 8))

# # После построения начальной модели, обновляем параметры с поступлением новых данных

# hist = []

# for t, y_value in enumerate(time_series, start=1500):
#     new_t = t + 1
#     new_y = y_value
    
#     # Обновление апостериорных распределений
#     posterior_theta_mean = (prior_theta_mean * prior_theta_std**2 + new_y * np.exp(-beta * new_t) * theta) / (prior_theta_std**2 + np.exp(-2 * beta * new_t))
#     posterior_theta_std = np.sqrt((prior_theta_std**2 * np.exp(2 * beta * new_t)) / (prior_theta_std**2 + np.exp(2 * beta * new_t)))
#     posterior_beta_mean = (prior_beta_mean * prior_beta_std**2 + (new_y - phi - theta * np.exp(-prior_beta_mean * new_t)) * new_t) / (prior_beta_std**2 + new_t**2)
#     posterior_beta_std = np.sqrt(prior_beta_std**2 / (prior_beta_std**2 + new_t**2))

#     # Генерация новых значений параметров из апостериорных распределений
#     theta = np.random.lognormal(posterior_theta_mean, posterior_theta_std)
#     beta = np.random.normal(posterior_beta_mean, posterior_beta_std)

#     # Добавление значений параметров в массивы
#     theta_values.append(theta)
#     beta_values.append(beta)

#     # Пересчет функции y(t) с новыми параметрами
#     new_y_value = y(new_t, phi, theta, beta)

#     print("Time:", new_t)
#     print("Updated theta:", theta)
#     print("Updated beta:", beta)
#     print("New y value:", new_y_value)

#     hist.append(new_y_value)

#     # Построение графиков плотности распределения параметров
#     fig, axes = plt.subplots(2, 1, figsize=(10, 8))
#     # Плотность распределения параметра theta
#     sns.kdeplot(theta_values, ax=axes[0])
#     # axes[0].hist(theta_values, bins=50, density=True, alpha=0.6, color='b')
#     axes[0].set_title('Density Plot of Parameter Theta')
#     axes[0].set_xlabel('Theta')
#     axes[0].set_ylabel('Density')

#     # Плотность распределения параметра beta
#     sns.kdeplot(beta_values, ax=axes[1])
#     # axes[1].hist(beta_values, bins=50, density=True, alpha=0.6, color='r')
#     axes[1].set_title('Density Plot of Parameter Beta')
#     axes[1].set_xlabel('Beta')
#     axes[1].set_ylabel('Density')

# plt.tight_layout()
# plt.show()


Подсчёт числа аномалий (граница - параметр функции). Экспоненциальный закон снижения остаточного ресурса.

2 варианта:
1. простое пересечение границы (в данных это была бы уставка)
2. автоэнкодер

Параметрическая граница

In [ ]:
def count_anomalies(data, thresh):
    """
    thresh: Максимальное допустимое значение параметра модели
    """
    return np.cumsum(data > thresh)

In [ ]:
time_series = exponential_moving_average(abs(compressed[:, 1]), window=100)
sns.lineplot(count_anomalies(time_series, time_series.mean()))

Функции нужно будет вынести в пакет

In [ ]:
# Получение данных каждые N минут
# Проверка превышения порога
# Если превышен: v_i = v_{i-1} + 1
# Иначе v_i = v_{i-1} + 0 <=> v_i = v_{i-1}
# Тоже фитить экспоненту?
# Или как-то статистически подобрать граничный параметр

xdata = np.arange(5)
ydata = exponential_moving_average(count_anomalies(time_series, time_series.mean()), window=100)[:1]
popt, pcov = curve_fit(fit_exp_custom, xdata, ydata)
popt2, pcov2 = curve_fit(fit_exp_classic, xdata, ydata)


In [ ]:
xdata_apriory = np.arange(1500, 3260)
y_true =  exponential_moving_average(count_anomalies(time_series, time_series.mean()), window=100)[xdata_apriory]
y_pred = fit_exp_custom(xdata_apriory, *popt)
y_pred2 = fit_exp_classic(xdata_apriory, *popt2)

plt.plot(y_true, label='True')
# plt.plot(y_pred, label='Pred1')
plt.plot(y_pred2, label='Pred2')
plt.legend()
plt.title('Anomaly prediction')

In [ ]:
def train(logger, x, y, func, start=5, end=3260):
    x_start = x[:start]
    y_start = y[:start]
    popt, pcov = curve_fit(func, x_start, y_start)

    init_pred = func(x[start:], *popt)

    logger.info('Started')

    logger.info(f'\nINIT LOSSES:\nMSE={mean_squared_error(y[start:], init_pred)},\nRMSE={mean_squared_error(y[start:], init_pred, squared=False)},\nMAE={mean_absolute_error(y[start:], init_pred)}')
    logger.info(f'INIT FUNC PARAMS: {popt}')

    for i in range(start + 1, end - 600):
        try:
            inner_popt, _ = curve_fit(func, x[:i], y[:i])
        except RuntimeError as RE:
            logger.info(f'\n{i} EPOCH LOSS:\nCAN NOT SOLVE')
            continue
        pred = func(x[i:], *inner_popt)

        # Определение запаздывания
        tick = - 1

        for j in range(0, xdata.shape[0] - 1):
            delta = abs(fit_exp_weird(xdata[:3420], *popt)[j] - 1499.2230827999854)
            if delta < 1:
                tick = j
                break

        logger.info(f'\n{i} EPOCH LOSS:\nMSE={mean_squared_error(y[i:], pred)},\nRMSE={mean_squared_error(y[i:], pred, squared=False)},\nMAE={mean_absolute_error(y[i:], pred)}\nRUL={3360 - tick, tick}')

        popt = np.mean( np.array([popt,inner_popt]), axis=0)
        logger.info(f'FUNC PARAMS: {popt}')
    
    return popt

In [ ]:
logger = logging.getLogger('logger_exp')
logging.basicConfig(filename='train_anomaly.log', level=logging.INFO)

xdata = np.arange(exponential_moving_average(count_anomalies(time_series, time_series.mean()), window=100).shape[0])
params = train(logger, xdata, exponential_moving_average(count_anomalies(time_series, time_series.mean()), window=100), fit_exp_custom)

logging.shutdown()

In [ ]:
logging.shutdown()

In [ ]:
sns.lineplot(ydata)
sns.lineplot(fit_exp_weird(xdata[:3420], *params))
sns.lineplot(exponential_moving_average(count_anomalies(time_series, time_series.mean()), window=100))

In [ ]:
exponential_moving_average(count_anomalies(time_series, time_series.mean()), window=100)[-1]

In [ ]:
exponential_moving_average(count_anomalies(time_series, time_series.mean()), window=100)

In [ ]:
for i in range(0, 3500):
    delta = abs(fit_exp_weird(xdata[:3420], *params)[i] - 1499.2230827999854)
    if delta < .1:
        print(f'Tick: {i}, dt = {3360 - i}')
        break

54 Часа отставания.  
Проблема - граница аномалий подобрана мной (просто среднее).  
Параметр должен подбираться на месте в течение какого-то времени. При этом возможно следует подсчитывать аномалии с нескольких сигналов. В качестве порога могут выступать уставки.

Нужно добавить вычисление и пересчёт доверительного интервала (чтобы можно было говорить про уверенность предсказания).